# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available for Part 1: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed and outputs displayed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2 of the Assignment (Dr Lu will be in charge of releasing this part). 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [1]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x7fa451007c40>)

In [2]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [3]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [4]:
air_quality_full.sample(5)

CO(GT)  PT08.S1(CO)  NMHC(GT)   C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  \
8547     2.2       910.50      -200   5.267605         776.75    310.4   
4530  -200.0      1361.25      -200  21.000300        1324.00   -200.0   
4504  -200.0      1219.00      -200  11.774165        1041.75   -200.0   
3330  -200.0       925.75      -200   6.901101         852.50     94.0   
2126     4.3      1200.50      -200  24.966344        1428.25    285.0   

      PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)       T         RH  \
8547        913.25    159.4        795.75       851.75   0.800  36.250000   
4530        540.25   -200.0       2044.25      1549.00  23.500  69.225002   
4504        699.00   -200.0       1749.50      1253.25  29.025  45.525001   
3330        894.00     90.0       1477.75       650.50  31.075  26.175001   
2126        664.25    118.0       2249.25      1377.50  21.700  44.150001   

            AH  
8547  0.238846  
4530  1.977573  
4504  1.797564  
3330  1.161923  
2126  1.132054

The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [5]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [6]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [7]:
np.random.seed(72812)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will design a random forests model and you are allowed to use scikit-learn in this section.

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

# 1. Using regularised ridge regression to predict air quality (10 marks)

**DO NOT USE scikit-learn or any other machine learning library for the questions on this section. You are meant to write Python code from scratch. You can use Pandas and Numpy. Using scikit-learn or any other machine learning library for the questions in this section will give ZERO marks. No excuse will be accepted.**

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [8]:
import math
data_training_unproc=data_training_unproc.replace(-200,math.nan)
train_mean=data_training_unproc.mean()
data_training_unproc= data_training_unproc.fillna(train_mean)

In [9]:
train_mean

CO(GT)              2.142200
PT08.S1(CO)      1110.276807
C6H6(GT)           10.216213
PT08.S2(NMHC)     944.811296
NOx(GT)           252.904822
PT08.S3(NOx)      829.094922
NO2(GT)           114.106320
PT08.S4(NO2)     1445.240820
PT08.S5(O3)      1040.688086
T                  17.815358
RH                 48.933293
AH                  0.991623
dtype: float64

In [10]:
train_std=data_training_unproc.std()
x=(data_training_unproc-train_mean)
Y=x/train_std

In [11]:
Y

CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
5804 -0.853940    -1.394397 -0.871179      -0.928857 -0.950910      0.899529   
8381 -0.853940    -0.821040 -1.044098      -1.223274 -0.612166      0.659885   
1240 -0.029010     0.043081 -0.197314      -0.054294 -0.648202      0.225345   
716   0.039734     0.000000  0.000000       0.000000 -0.369519      0.000000   
6345 -0.922684    -0.998536 -1.136956      -1.406681  0.000000      1.410635   
...        ...          ...       ...            ...       ...           ...   
2191 -0.991428    -1.083780 -0.687769      -0.658572 -1.085447      0.937315   
7321  0.383454     0.518348 -0.124283       0.025826  0.859090     -0.748145   
6917  1.414616     1.241173  0.926733       1.018157  2.340436     -1.115068   
4600 -0.510219    -0.866582 -0.502614      -0.414351 -0.206153      0.476921   
7179  0.727175     0.000000  0.000000       0.000000  1.232430      0.000000   

       NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
5804 -1.871431     -1.172923    -1.316594 -0.033484 -0.901586 -0.818002  
8381 -0.072352     -1.340071    -1.085973 -1.544168  1.665573 -0.860504  
1240 -0.533273      0.602195    -0.177390  0.433560 -0.169169  0.458759  
716   0.103945      0.000000     0.000000  0.000000  0.000000  0.000000  
6345  0.000000     -0.771478    -1.024053 -0.486113  2.007367  0.742731  
...        ...           ...          ...       ...       ...       ...  
2191 -1.531581      0.238704    -1.235087  0.698794 -0.195802  0.865326  
7321  1.083136     -0.324779     0.269319 -0.307367  0.342783 -0.140444  
6917  1.612027     -0.472949     2.527508 -1.345242  0.339824 -1.195813  
4600 -0.660717     -0.354705    -0.790905  0.874656 -0.837962  0.305003  
7179  1.034283      0.000000     0.000000  0.000000  0.000000  0.000000  

[5372 rows x 12 columns]

In [166]:
#Training set after imputing and standardization

Y.head()

CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
5804 -0.853940    -1.394397 -0.871179      -0.928857 -0.950910      0.899529   
8381 -0.853940    -0.821040 -1.044098      -1.223274 -0.612166      0.659885   
1240 -0.029010     0.043081 -0.197314      -0.054294 -0.648202      0.225345   
716   0.039734     0.000000  0.000000       0.000000 -0.369519      0.000000   
6345 -0.922684    -0.998536 -1.136956      -1.406681  0.000000      1.410635   

       NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
5804 -1.871431     -1.172923    -1.316594 -0.033484 -0.901586 -0.818002  
8381 -0.072352     -1.340071    -1.085973 -1.544168  1.665573 -0.860504  
1240 -0.533273      0.602195    -0.177390  0.433560 -0.169169  0.458759  
716   0.103945      0.000000     0.000000  0.000000  0.000000  0.000000  
6345  0.000000     -0.771478    -1.024053 -0.486113  2.007367  0.742731

## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [14]:
yTrain=Y['CO(GT)']
XTrain=Y.drop(['CO(GT)'],axis=1)

In [15]:
XTrain


PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)   NO2(GT)  \
5804    -1.394397 -0.871179      -0.928857 -0.950910      0.899529 -1.871431   
8381    -0.821040 -1.044098      -1.223274 -0.612166      0.659885 -0.072352   
1240     0.043081 -0.197314      -0.054294 -0.648202      0.225345 -0.533273   
716      0.000000  0.000000       0.000000 -0.369519      0.000000  0.103945   
6345    -0.998536 -1.136956      -1.406681  0.000000      1.410635  0.000000   
...           ...       ...            ...       ...           ...       ...   
2191    -1.083780 -0.687769      -0.658572 -1.085447      0.937315 -1.531581   
7321     0.518348 -0.124283       0.025826  0.859090     -0.748145  1.083136   
6917     1.241173  0.926733       1.018157  2.340436     -1.115068  1.612027   
4600    -0.866582 -0.502614      -0.414351 -0.206153      0.476921 -0.660717   
7179     0.000000  0.000000       0.000000  1.232430      0.000000  1.034283   

      PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
5804     -1.172923    -1.316594 -0.033484 -0.901586 -0.818002  
8381     -1.340071    -1.085973 -1.544168  1.665573 -0.860504  
1240      0.602195    -0.177390  0.433560 -0.169169  0.458759  
716       0.000000     0.000000  0.000000  0.000000  0.000000  
6345     -0.771478    -1.024053 -0.486113  2.007367  0.742731  
...            ...          ...       ...       ...       ...  
2191      0.238704    -1.235087  0.698794 -0.195802  0.865326  
7321     -0.324779     0.269319 -0.307367  0.342783 -0.140444  
6917     -0.472949     2.527508 -1.345242  0.339824 -1.195813  
4600     -0.354705    -0.790905  0.874656 -0.837962  0.305003  
7179      0.000000     0.000000  0.000000  0.000000  0.000000  

[5372 rows x 11 columns]

In [16]:
yTrain.head()

5804   -0.853940
8381   -0.853940
1240   -0.029010
716     0.039734
6345   -0.922684
Name: CO(GT), dtype: float64

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**).
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**).

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations (**1 mark**).

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [26]:
data_val_unproc=data_val_unproc.replace(-200,train_mean)
x_val=(data_val_unproc-train_mean)
x_val=x_val/train_std

In [38]:
reg = np.logspace(-6, -1, num = 5)
reg

array([1.00000000e-06, 1.77827941e-05, 3.16227766e-04, 5.62341325e-03,
       1.00000000e-01])

In [39]:
lr = np.logspace(-4, 0, num = 5)
lr

array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00])

In [40]:
S = np.linspace(200, 800, num = 5, dtype = int)
S

array([200, 350, 500, 650, 800])

In [21]:
XTrain.reset_index(drop=True,inplace=True)

In [22]:
XTrain

PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)   NO2(GT)  \
0       -1.394397 -0.871179      -0.928857 -0.950910      0.899529 -1.871431   
1       -0.821040 -1.044098      -1.223274 -0.612166      0.659885 -0.072352   
2        0.043081 -0.197314      -0.054294 -0.648202      0.225345 -0.533273   
3        0.000000  0.000000       0.000000 -0.369519      0.000000  0.103945   
4       -0.998536 -1.136956      -1.406681  0.000000      1.410635  0.000000   
...           ...       ...            ...       ...           ...       ...   
5367    -1.083780 -0.687769      -0.658572 -1.085447      0.937315 -1.531581   
5368     0.518348 -0.124283       0.025826  0.859090     -0.748145  1.083136   
5369     1.241173  0.926733       1.018157  2.340436     -1.115068  1.612027   
5370    -0.866582 -0.502614      -0.414351 -0.206153      0.476921 -0.660717   
5371     0.000000  0.000000       0.000000  1.232430      0.000000  1.034283   

      PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
0        -1.172923    -1.316594 -0.033484 -0.901586 -0.818002  
1        -1.340071    -1.085973 -1.544168  1.665573 -0.860504  
2         0.602195    -0.177390  0.433560 -0.169169  0.458759  
3         0.000000     0.000000  0.000000  0.000000  0.000000  
4        -0.771478    -1.024053 -0.486113  2.007367  0.742731  
...            ...          ...       ...       ...       ...  
5367      0.238704    -1.235087  0.698794 -0.195802  0.865326  
5368     -0.324779     0.269319 -0.307367  0.342783 -0.140444  
5369     -0.472949     2.527508 -1.345242  0.339824 -1.195813  
5370     -0.354705    -0.790905  0.874656 -0.837962  0.305003  
5371      0.000000     0.000000  0.000000  0.000000  0.000000  

[5372 rows x 11 columns]

In [24]:
XTrain_new = XTrain.to_numpy()
XTrain_new = np.hstack((np.ones((len(XTrain_new),1)),XTrain_new))
yTrain_new = yTrain.to_frame() 
yTrain_new = yTrain_new.to_numpy()
yTrain_new = np.array(yTrain_new).flatten()

In [27]:
df_val_new = x_val

In [29]:
yVal = x_val['CO(GT)']
yVal

6270    1.345872
1235   -1.060172
5480   -0.853940
5977   -0.647707
9059   -1.128916
          ...   
9108   -1.060172
6985    0.933407
4060   -0.716451
2247   -0.166498
5603   -0.785196
Name: CO(GT), Length: 1151, dtype: float64

In [32]:
XVal = x_val.drop('CO(GT)',1)
XVal.reset_index(drop=True,inplace=True)
XVal

PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)   NO2(GT)  \
0        1.815697  0.920985       1.013330  1.927216     -0.322554  0.613719   
1       -1.178366 -1.052367      -1.238719 -1.099861      1.496151 -1.743987   
2       -1.026561 -0.794748      -0.812055 -0.677031      0.192530 -1.489100   
3       -1.279959 -0.990028      -1.125779 -0.220567      3.854800 -0.320867   
4       -0.627197 -1.159860      -1.455912 -0.841359      0.246227 -1.083405   
...           ...       ...            ...       ...           ...       ...   
1146    -0.607346 -1.058516      -1.250302 -0.582856      0.114970 -0.701074   
1147     1.458371  0.518963       0.661960  1.427508     -1.097169  0.677441   
1148     0.000000  0.000000       0.000000 -0.888446      0.000000 -0.915604   
1149     0.059429  0.400819       0.552881 -0.460812     -0.329514 -0.469551   
1150    -0.878259 -0.663238      -0.624787 -0.292641      0.111986 -0.809401   

      PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
0         0.246003     2.122500 -0.561070  1.464343  0.263937  
1        -0.032089    -1.257833 -0.134388  1.076680  0.685478  
2        -0.270766    -0.271534 -0.151686  1.570876  1.049959  
3        -1.364887    -1.273629 -1.085773  0.399009 -0.953930  
4        -0.881693    -0.849034 -0.321782  0.942034  0.276696  
...            ...          ...       ...       ...       ...  
1146     -0.745202    -0.880626 -0.474581  1.721799  0.572672  
1147      0.251113     0.829128 -0.633145  1.564958  0.216077  
1148      0.000000     0.000000  0.000000  0.000000  0.000000  
1149      1.003640     0.379259  0.713209 -0.729948  0.216962  
1150     -0.229162    -0.545752 -0.128622  1.820934  1.296894  

[1151 rows x 11 columns]

In [33]:
XVal_new = XVal.to_numpy()
XVal_new = np.hstack((np.ones((len(XVal_new),1)),XVal_new))
yVal_new = yVal.to_frame() 
yVal_new = yVal_new.to_numpy()
yVal_new = np.array(yVal_new).flatten()

In [34]:
def shuffle_matrices_same_order(x,y):
    index = np.arange(len(x))
    np.random.shuffle(index)
    x = x[index,:]
    y = y[index]
    return x,y

In [35]:
def gradientDescent_minibatch(x_train,y_train,x_test,y_test,beta,alpha,Lamda,BatchSize,numIterations):
    Train_loss = []
    Train_RMSE = []
    Test_loss = []
    Test_RMSE = []
    for i in range(0,numIterations):
        x_train,y_train = shuffle_matrices_same_order(x_train,y_train)
        for j in range(np.int(np.ceil(len(x_train)/BatchSize))):
            data_range = range(j*BatchSize,min((j+1)*BatchSize,len(x_train)))
            
            y_hat = np.dot(x_train[data_range,:], beta)
            loss = y_train[data_range] - y_hat
            cost = np.sum(loss ** 2)
            gradient = -2*np.dot(x_train[data_range,:].transpose(),loss)
            beta[0] = beta[0] - alpha*gradient[0]
            for k in range(1,len(beta)):
                beta[k] = beta[k]*(1-2*alpha*Lamda) - alpha*gradient[k]
        y_hat = np.dot(x_train,beta); loss =  y_train - y_hat; cost = np.sum(loss ** 2)
        Train_loss.append(np.sum(abs(loss)))
        Train_RMSE.append((cost/len(x_train))**0.5)
        
        y_hat = np.dot(x_test,beta); loss = y_test - y_hat; cost = np.sum(loss**2)
        
        Test_loss.append(np.sum(abs(loss)))
        Test_RMSE.append((cost/len(x_test))**0.5)
        
    return beta, Train_RMSE, Test_RMSE

In [41]:
reg = reg.tolist()
lr = lr.tolist()
S = S.tolist()


In [42]:
numIterations = 200
Lamda = reg
alpha = lr
BatchSize = S
beta = np.ones(XTrain_new.shape[1])
results = pd.DataFrame()

In [43]:
for i in alpha:
    for j in Lamda:
        for k in BatchSize:
            alpha_curr = i
            Lamda_curr = j
            BatchSize_curr = k
            resuts_int = gradientDescent_minibatch(XTrain_new,
                                              yTrain_new,
                                              XVal_new,
                                              yVal_new,
                                              beta,
                                              alpha_curr,
                                              Lamda_curr,
                                              BatchSize_curr,
                                              numIterations)
            results['Train_Data_RMSE values: alpha='+str(i)+'Lamda='+str(j)+'BatchSize'+str(k)] = resuts_int[1]
            results['Validation_Data_RMSE values: alpha='+str(i)+'Lamda='+str(j)+'BatchSize'+str(k)] = resuts_int[2]

<ipython-input-35-b914c09c85c6>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  for j in range(np.int(np.ceil(len(x_train)/BatchSize))):
/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
<ipython-input-35-b914c09c85c6>:13: RuntimeWarning: overflow encountered in square
  cost = np.sum(loss ** 2)
<ipython-input-35-b914c09c85c6>:18: RuntimeWarning: overflow encountered in square
  y_hat = np.dot(x_train,

In [48]:
results

Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize200  \
0                                           0.909260   
1                                           0.671103   
2                                           0.561089   
3                                           0.495842   
4                                           0.455733   
..                                               ...   
195                                         0.338629   
196                                         0.338611   
197                                         0.339574   
198                                         0.338647   
199                                         0.338615   

     Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSize200  \
0                                             0.879495      
1                                             0.657018      
2                                             0.554451      
3                                             0.494483      
4                                             0.454868      
..                                                 ...      
195                                           0.350650      
196                                           0.350363      
197                                           0.350359      
198                                           0.350273      
199                                           0.350483      

     Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize350  \
0                                           0.338857   
1                                           0.338584   
2                                           0.339359   
3                                           0.338646   
4                                           0.338600   
..                                               ...   
195                                         0.338699   
196                                         0.338556   
197                                         0.338623   
198                                         0.338755   
199                                         0.338575   

     Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSize350  \
0                                             0.350881      
1                                             0.350245      
2                                             0.350192      
3                                             0.350252      
4                                             0.350056      
..                                                 ...      
195                                           0.350681      
196                                           0.350477      
197                                           0.350060      
198                                           0.350034      
199                                           0.350489      

     Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize500  \
0                                           0.339211   
1                                           0.338810   
2                                           0.338565   
3                                           0.338786   
4                                           0.339296   
..                                               ...   
195                                         0.338967   
196                                         0.338863   
197                                         0.338647   
198                                         0.338580   
199                                         0.338685   

     Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSize500  \
0                                             0.350219      
1                                             0.351091      
2                                             0.350561      
3                                             0.350963      
4                                             0.351949      
..                                                 ...      
195                                           0.351508      
196                             

In [67]:
minimum = results.apply(lambda x:x.min())


In [68]:
minimum = minimum.to_frame()
minimum

0
Parameters                                                  
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize200     0.338598
Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSi...  0.349806
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize350     0.338550
Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSi...  0.349863
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize500     0.338549
...                                                      ...
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize500          NaN
Train_RMSE: alpha=1.0Lamda=0.1BatchSize650               NaN
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize650          NaN
Train_RMSE: alpha=1.0Lamda=0.1BatchSize800               NaN
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize800          NaN

[250 rows x 1 columns]

In [69]:
minimum.columns=['RMSE']
minimum.index.name = 'Parameters'
minimum

RMSE
Parameters                                                  
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize200     0.338598
Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSi...  0.349806
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize350     0.338550
Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSi...  0.349863
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize500     0.338549
...                                                      ...
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize500          NaN
Train_RMSE: alpha=1.0Lamda=0.1BatchSize650               NaN
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize650          NaN
Train_RMSE: alpha=1.0Lamda=0.1BatchSize800               NaN
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize800          NaN

[250 rows x 1 columns]

In [59]:
minimum

Parameters
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize200         0.338598
Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSize200    0.349806
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize350         0.338550
Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSize350    0.349863
Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize500         0.338549
                                                          ...   
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize500              NaN
Train_RMSE: alpha=1.0Lamda=0.1BatchSize650                   NaN
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize650              NaN
Train_RMSE: alpha=1.0Lamda=0.1BatchSize800                   NaN
Validation_RMSE: alpha=1.0Lamda=0.1BatchSize800              NaN
Length: 250, dtype: float64

In [71]:
minimum.reset_index(level=0, inplace=True)

In [73]:
Val_min = pd.DataFrame()
Val_min = minimum.sort_values(by='RMSE')
Val_min.head(26)

Parameters      RMSE
48      Train_RMSE: alpha=0.0001Lamda=0.1BatchSize800  0.338547
30  Train_RMSE: alpha=0.0001Lamda=0.00562341325190...  0.338547
46      Train_RMSE: alpha=0.0001Lamda=0.1BatchSize650  0.338548
40      Train_RMSE: alpha=0.0001Lamda=0.1BatchSize200  0.338548
16  Train_RMSE: alpha=0.0001Lamda=1.77827941003892...  0.338548
38  Train_RMSE: alpha=0.0001Lamda=0.00562341325190...  0.338548
6     Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize650  0.338548
10  Train_RMSE: alpha=0.0001Lamda=1.77827941003892...  0.338548
24  Train_RMSE: alpha=0.0001Lamda=0.00031622776601...  0.338548
14  Train_RMSE: alpha=0.0001Lamda=1.77827941003892...  0.338548
32  Train_RMSE: alpha=0.0001Lamda=0.00562341325190...  0.338548
12  Train_RMSE: alpha=0.0001Lamda=1.77827941003892...  0.338549
22  Train_RMSE: alpha=0.0001Lamda=0.00031622776601...  0.338549
18  Train_RMSE: alpha=0.0001Lamda=1.77827941003892...  0.338549
26  Train_RMSE: alpha=0.0001Lamda=0.00031622776601...  0.338549
34  Train_RMSE: alpha=0.0001Lamda=0.00562341325190...  0.338549
44      Train_RMSE: alpha=0.0001Lamda=0.1BatchSize500  0.338549
4     Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize500  0.338549
36  Train_RMSE: alpha=0.0001Lamda=0.00562341325190...  0.338550
20  Train_RMSE: alpha=0.0001Lamda=0.00031622776601...  0.338550
2     Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize350  0.338550
28  Train_RMSE: alpha=0.0001Lamda=0.00031622776601...  0.338550
42      Train_RMSE: alpha=0.0001Lamda=0.1BatchSize350  0.338551
8     Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize800  0.338551
0     Train_RMSE: alpha=0.0001Lamda=1e-06BatchSize200  0.338598
9   Validation_RMSE: alpha=0.0001Lamda=1e-06BatchS...  0.349802

In [74]:
print(Val_min['Parameters'].iloc[25])
print(Val_min['RMSE'].iloc[25])


Validation_RMSE: alpha=0.0001Lamda=1e-06BatchSize800
0.3498019711735793


### The best parameters in Validation data - 
#### alpha(learning rate): 0.0001, Lamda: 1.778279410038923e-05, Batch Size: 800

In [76]:
best_alpha = 0.0001
best_lamda = 1.778279410038923e-05
best_S = 800

## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [47]:
# The .csv version of the file has some typing issues, so we use the excel version
air_quality_full = pd.read_excel('AirQualityUCI.xlsx', usecols=range(2,15))

# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

# We now remove the columns (features) for which there are more that 20% of missing values
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

np.random.seed(72812)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data


In [77]:
df_Train = pd.merge(data_training_unproc,data_val_unproc,how='outer')
df_Train.shape

(6523, 12)

In [79]:
#Percentafge of misiing values:
round(100*((df_Train.iloc[:,:]==-200).sum())/len(df_Train.index), 2)

CO(GT)           0.00
PT08.S1(CO)      4.32
C6H6(GT)         4.32
PT08.S2(NMHC)    4.32
NOx(GT)          5.23
PT08.S3(NOx)     4.32
NO2(GT)          5.27
PT08.S4(NO2)     4.32
PT08.S5(O3)      4.32
T                4.32
RH               4.32
AH               4.32
dtype: float64

In [80]:
df_Train = df_Train.replace(-200,math.nan)
train_mean = df_Train.mean()
df_Train = df_Train.fillna(train_mean)
train_std = train_mean.std()
x = (df_Train - train_mean)
df_Train = x/train_std

In [82]:
df_Train.head()

CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
0  0.001369     0.279882  0.014246       0.538239 -0.052469     -0.400321   
1  0.001001     0.303798  0.010173       0.416363  0.200663     -0.543352   
2  0.003944     0.580661  0.016733       0.609524  1.000348     -0.599920   
3  0.001921     0.187748  0.011882       0.468333  0.346729     -0.452137   
4 -0.002678    -0.281663 -0.015443      -0.681893 -0.260899      0.149267   

    NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
0  0.025886      0.999385     0.431162  0.017102 -0.021817  0.000589  
1  0.041155      0.063936     0.646858 -0.007641  0.015987 -0.000174  
2  0.110509      0.065315     1.659112 -0.017391  0.017275 -0.000640  
3  0.025886      0.539786     0.482671  0.005512  0.017459  0.000784  
4 -0.065543     -0.674214    -0.919124 -0.010860  0.033188 -0.000111

### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**).

* Report the RMSE over the test data (**1 mark**).

#### Question 1.d Answer

In [83]:
data_test_unproc.head()

CO(GT)  PT08.S1(CO)   C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  \
6770     3.3      1324.50  16.371624        1191.00    403.0        563.50   
2196     1.2       949.00   8.731502         928.75    101.0        887.25   
1631     2.4      1106.25  13.400779        1097.00    135.0        901.75   
5518     5.8      1669.00  32.788498        1615.00    760.0        403.50   
7460     4.4      1354.75  18.148650        1243.75    643.7        536.00   

      NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)          T      RH        AH  
6770    130.0       1541.50       1389.0  13.350000  68.000  1.037552  
2196     78.0       1710.75        972.5  20.000000  56.625  1.309451  
1631    105.0       1552.00        941.0  26.724999  17.100  0.590941  
5518    166.0       2232.25       1963.0  24.025000  55.000  1.621115  
7460    211.5       1453.75       1714.5  10.600000  55.250  0.705801

In [84]:
#Percentafge of missing values:
round(100*((data_test_unproc.iloc[:,:]==-200).sum())/len(data_test_unproc.index), 2)

CO(GT)           0.00
PT08.S1(CO)      4.17
C6H6(GT)         4.17
PT08.S2(NMHC)    4.17
NOx(GT)          6.26
PT08.S3(NOx)     4.17
NO2(GT)          6.26
PT08.S4(NO2)     4.17
PT08.S5(O3)      4.17
T                4.17
RH               4.17
AH               4.17
dtype: float64

In [85]:
data_test_unproc = data_test_unproc.replace(-200,math.nan)
test_mean = data_test_unproc.mean()
data_test_unproc = data_test_unproc.fillna(test_mean)
test_mean

CO(GT)              2.135361
PT08.S1(CO)      1103.926035
C6H6(GT)           10.081782
PT08.S2(NMHC)     939.470686
NOx(GT)           248.285635
PT08.S3(NOx)      830.688728
NO2(GT)           114.036145
PT08.S4(NO2)     1441.165684
PT08.S5(O3)      1030.726806
T                  18.135645
RH                 48.767490
AH                  0.998079
dtype: float64

In [86]:
test_std = test_mean.std()
x1 = (data_test_unproc - test_mean)
data_test_unproc = x1/test_std
data_test_unproc.head()

CO(GT)  PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)  \
6770  0.002154     0.407938  0.011633       0.465188  0.286135     -0.494149   
2196 -0.001730    -0.286526 -0.002497      -0.019827 -0.272396      0.104607   
1631  0.000489     0.004298  0.006138       0.291341 -0.209515      0.131424   
5518  0.006778     1.045070  0.041995       1.249351  0.946385     -0.790060   
7460  0.004188     0.463884  0.014919       0.562746  0.731295     -0.545009   

       NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
6770  0.029524      0.185562     0.662605 -0.008851  0.035569  0.000073  
2196 -0.066647      0.498580    -0.107687  0.003448  0.014532  0.000576  
1631 -0.016712      0.204981    -0.165944  0.015885 -0.058567 -0.000753  
5518  0.096104      1.463062     1.724183  0.010892  0.011527  0.001152  
7460  0.180253      0.023274     1.264597 -0.013937  0.011989 -0.000541

In [87]:
df_Train.reset_index(drop=True,inplace=True)
df_Train_new = df_Train
yTrain = df_Train['CO(GT)']
yTrain

0       0.001369
1       0.001001
2       0.003944
3       0.001921
4      -0.002678
          ...   
6518    0.002105
6519   -0.001022
6520   -0.000287
6521    0.001369
6522   -0.001942
Name: CO(GT), Length: 6523, dtype: float64

In [88]:
XTrain = df_Train.drop('CO(GT)',1)
XTrain

PT08.S1(CO)  C6H6(GT)  PT08.S2(NMHC)   NOx(GT)  PT08.S3(NOx)   NO2(GT)  \
0        0.279882  0.014246       0.538239 -0.052469     -0.400321  0.025886   
1        0.303798  0.010173       0.416363  0.200663     -0.543352  0.041155   
2        0.580661  0.016733       0.609524  1.000348     -0.599920  0.110509   
3        0.187748  0.011882       0.468333  0.346729     -0.452137  0.025886   
4       -0.281663 -0.015443      -0.681893 -0.260899      0.149267 -0.065543   
...           ...       ...            ...       ...           ...       ...   
6518     0.126274  0.008404       0.361175 -0.065347     -0.207620  0.024046   
6519     0.041651 -0.006790      -0.199451 -0.282423      0.358985 -0.034822   
6520     0.000000  0.000000       0.000000 -0.234592      0.000000 -0.016425   
6521     0.551687  0.015200       0.565833  0.140691     -0.553470 -0.031142   
6522     0.059587 -0.012999      -0.517707 -0.291621      1.220849 -0.060576   

      PT08.S4(NO2)  PT08.S5(O3)         T        RH        AH  
0         0.999385     0.431162  0.017102 -0.021817  0.000589  
1         0.063936     0.646858 -0.007641  0.015987 -0.000174  
2         0.065315     1.659112 -0.017391  0.017275 -0.000640  
3         0.539786     0.482671  0.005512  0.017459  0.000784  
4        -0.674214    -0.919124 -0.010860  0.033188 -0.000111  
...            ...          ...       ...       ...       ...  
6518      0.890388     0.266975  0.011813 -0.004709  0.000722  
6519      0.103028    -0.236623 -0.010584  0.040454  0.000007  
6520      0.000000     0.000000  0.000000  0.000000  0.000000  
6521      0.646177     0.715384  0.013331  0.003340  0.001100  
6522     -0.195452    -0.581093 -0.013804  0.019299 -0.000453  

[6523 rows x 11 columns]

In [89]:
XTrain_new = XTrain.to_numpy()
XTrain_new = np.hstack((np.ones((len(XTrain_new),1)),XTrain_new))
yTrain_new = yTrain.to_frame() 
yTrain_new = yTrain_new.to_numpy()
yTrain_new = np.array(yTrain_new).flatten()

In [90]:
numIterations = 200
Lamda = best_lamda
alpha = best_alpha
BatchSize = best_S
beta = np.ones(XTrain_new.shape[1])
results_train = pd.DataFrame()

In [91]:
data_test_unproc.reset_index(drop=True,inplace=True)

In [92]:
data_test_unproc_new = data_test_unproc
yTest = data_test_unproc['CO(GT)']
XTest = data_test_unproc.drop('CO(GT)',1)

In [94]:
XTest_new = XTest.to_numpy()
XTest_new = np.hstack((np.ones((len(XTest_new),1)),XTest_new))
yTest_new = yTest.to_frame() 
yTest_new = yTest_new.to_numpy()
yTest_new = np.array(yTest_new).flatten()

In [95]:
alpha_curr = alpha
Lamda_curr = Lamda
BatchSize_curr = BatchSize
resuts_train = gradientDescent_minibatch(XTrain_new,
                                              yTrain_new,
                                              XTest_new,
                                              yTest_new,
                                              beta,
                                              alpha_curr,
                                              Lamda_curr,
                                              BatchSize_curr,
                                              numIterations)

<ipython-input-35-b914c09c85c6>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  for j in range(np.int(np.ceil(len(x_train)/BatchSize))):


### RMSE on test dataset

In [96]:
resuts_train[2]

[0.5669317018573516,
 0.39052932066175994,
 0.35165963540335965,
 0.3225283076446056,
 0.2965100889150766,
 0.27289063341510905,
 0.2513392355682741,
 0.23171424789090966,
 0.21369313720593125,
 0.19724493651543723,
 0.18221838538838714,
 0.1685010098010758,
 0.155973549154713,
 0.1445166035366247,
 0.13405546081983188,
 0.1245228846125955,
 0.11584870810543106,
 0.10795296895333245,
 0.10077816984902871,
 0.09426335981922591,
 0.08836745217340815,
 0.0830011855295867,
 0.07815905962139191,
 0.0738055133325019,
 0.06984443618049077,
 0.06629324404020073,
 0.06309583271089375,
 0.060222145302050445,
 0.05764524555494823,
 0.05533840330442666,
 0.05326013614817712,
 0.05140972855139714,
 0.049744668550582335,
 0.04825521268528362,
 0.04692216196158958,
 0.04572525302350354,
 0.044649698792144484,
 0.0436838343151965,
 0.04281509253675589,
 0.04202570544586242,
 0.0413123087830712,
 0.04066861693028983,
 0.04008460470781177,
 0.03953879929526466,
 0.039046561402985264,
 0.0385915674284678

### RMSE on test dataset - 0.024113

# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [115]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

import pandas as pd 
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import math

In [139]:
# The .csv version of the file has some typing issues, so we use the excel version
air_quality_full = pd.read_excel('AirQualityUCI.xlsx', usecols=range(2,15))

# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

# We now remove the columns (features) for which there are more that 20% of missing values
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

np.random.seed(72812)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data


In [140]:
eda = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())])

In [141]:
df_new_hold = data_training_unproc
df_val_hold = data_val_unproc
df_test_hold = data_test_unproc
y_train_1 = data_training_unproc['CO(GT)']
X_train_1 = data_training_unproc.drop('CO(GT)',1)
y_val_1 = data_val_unproc['CO(GT)']
x_val_1 = data_val_unproc.drop('CO(GT)',1)

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

#### Question 2.b Answer

In [143]:
X_train_2 = eda.fit_transform(X_train_1)
x_val_2 = eda.transform(x_val_1)

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

#### Question 2.c Answer

In [144]:
X_train_full = np.concatenate((X_train_2, x_val_2), axis = 0)
Y_train_full = np.concatenate((y_train_1, y_val_1), axis = 0)

In [145]:
# Create a list where train data indices are -1 and validation data indices are 0
split_index = [-1 if x in X_train_2 else 0 for x in x_val_2]

# Use the list to create PredefinedSplit
pds = PredefinedSplit(test_fold = split_index)

In [146]:
pds.unique_folds

array([], dtype=int64)

In [124]:
X_train_full = np.concatenate((X_train_2, x_val_2), axis = 0)
Y_train_full = np.concatenate((y_train_1, y_val_1), axis = 0)

n_estimators = np.linspace(40, 400, num = 5, dtype = int)


max_samples = np.linspace(600, 4000, num = 5, dtype = int)


max_features = np.linspace(3, 11, num = 5, dtype = int)


In [125]:
print("estimators-",n_estimators,",max samples-",max_samples,",max features-",max_features)

estimators- [ 40 130 220 310 400] ,max samples- [ 600 1450 2300 3150 4000] ,max features- [ 3  5  7  9 11]


In [147]:
# hyperparameter space
params = {'max_features': max_features,
          'n_estimators': n_estimators,
          'max_samples': max_samples}


model = GridSearchCV(estimator=RandomForestRegressor(), cv=2, param_grid=params, scoring='neg_mean_squared_error')

In [148]:
# fit model
model.fit(X_train_full, Y_train_full)# fit model

/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 343, in fit
    n_samples_bootstrap = _get_n_samples_bootstrap(
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 104, in _get_n_samples_bootstrap
    raise ValueError(msg.format(n_samples, max_samples))
ValueError: `max_samples` must be in range 1 to 3261 but got value 4000

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/mahimpatil/opt/anaconda3/lib/python

/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 343, in fit
    n_samples_bootstrap = _get_n_samples_bootstrap(
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 104, in _get_n_samples_bootstrap
    raise ValueError(msg.format(n_samples, max_samples))
ValueError: `max_samples` must be in range 1 to 3261 but got value 4000

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/mahimpatil/opt/anaconda3/lib/python

/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 343, in fit
    n_samples_bootstrap = _get_n_samples_bootstrap(
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 104, in _get_n_samples_bootstrap
    raise ValueError(msg.format(n_samples, max_samples))
ValueError: `max_samples` must be in range 1 to 3261 but got value 4000

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/mahimpatil/opt/anaconda3/lib/python

/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 343, in fit
    n_samples_bootstrap = _get_n_samples_bootstrap(
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 104, in _get_n_samples_bootstrap
    raise ValueError(msg.format(n_samples, max_samples))
ValueError: `max_samples` must be in range 1 to 3261 but got value 4000

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/mahimpatil/opt/anaconda3/lib/python

/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 343, in fit
    n_samples_bootstrap = _get_n_samples_bootstrap(
  File "/Users/mahimpatil/opt/anaconda3/lib/python3.8/site-packages/sklearn/ensemble/_forest.py", line 104, in _get_n_samples_bootstrap
    raise ValueError(msg.format(n_samples, max_samples))
ValueError: `max_samples` must be in range 1 to 3261 but got value 4000

  warnings.warn("Estimator fit failed. The score on this train-test"
/Users/mahimpatil/opt/anaconda3/lib/python

GridSearchCV(cv=2, estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 3,  5,  7,  9, 11]),
                         'max_samples': array([ 600, 1450, 2300, 3150, 4000]),
                         'n_estimators': array([ 40, 130, 220, 310, 400])},
             scoring='neg_mean_squared_error')

In [150]:
#Best print best hyperparameters:
print('Best Hyperparameters:',model.best_params_)

Best Hyperparameters: {'max_features': 3, 'max_samples': 3150, 'n_estimators': 400}


In [151]:
print("Best neg_mean_squared_error: ", model.best_score_)

Best neg_mean_squared_error:  -0.18528347181420762


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**).

* Apply the created preprocessing pipeline to the test data (**1 mark**).

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**).

* Compute the RMSE over the test data and report the result (**1 mark**).

#### Question 2.d Answer

In [152]:
print(df_new_hold.shape)
print(df_val_hold.shape)

(5372, 12)
(1151, 12)


In [153]:
df_Train_new = pd.merge(df_new_hold,df_val_hold,how='outer')
df_Train_new.shape

(6523, 12)

In [154]:
#Percentafge of misiing values:
round(100*((df_Train_new.iloc[:,:]==-200).sum())/len(df_Train_new.index), 2)

CO(GT)           0.00
PT08.S1(CO)      4.49
C6H6(GT)         4.49
PT08.S2(NMHC)    4.49
NOx(GT)          5.27
PT08.S3(NOx)     4.49
NO2(GT)          5.32
PT08.S4(NO2)     4.49
PT08.S5(O3)      4.49
T                4.49
RH               4.49
AH               4.49
dtype: float64

In [155]:
yTrain_new = df_Train_new['CO(GT)']
X_Train_new = df_Train_new.drop('CO(GT)',1)
X_Train_new.reset_index(drop=True,inplace=True)

In [157]:
df_test_hold.shape

(1151, 12)

In [158]:
#Percentafge of misiing values:
round(100*((df_test_hold.iloc[:,:]==-200).sum())/len(df_test_hold.index), 2)

CO(GT)           0.00
PT08.S1(CO)      3.21
C6H6(GT)         3.21
PT08.S2(NMHC)    3.21
NOx(GT)          5.99
PT08.S3(NOx)     3.21
NO2(GT)          5.99
PT08.S4(NO2)     3.21
PT08.S5(O3)      3.21
T                3.21
RH               3.21
AH               3.21
dtype: float64

In [159]:
df_test_hold.reset_index(drop=True,inplace=True)
df_test_hold.shape

(1151, 12)

In [160]:
yTest_new = df_test_hold['CO(GT)']
X_Test_new = df_test_hold.drop('CO(GT)',1)

In [161]:
X_Train_full = eda.fit_transform(X_Train_new)
x_Test_full = eda.transform(X_Test_new)

In [162]:
# hyperparameter space
params_best = { 'max_features': 5,
          'n_estimators': 400,
          'max_samples': 600}

# create gridsearch object
model_final = RandomForestRegressor(n_estimators=params_best['n_estimators'],max_samples=params_best['max_samples'],max_features=params_best['max_features'])

In [163]:
# fit model
model_final.fit(X_Train_full, yTrain_new)

RandomForestRegressor(max_features=5, max_samples=600, n_estimators=400)

In [164]:
#Prodictions:
pred = model_final.predict(x_Test_full)
error_test = np.sqrt(mean_squared_error(yTest_new, pred))

In [165]:
print('RMSE on the Test dataset is:', error_test)

RMSE on the Test dataset is: 0.42959794288679404
